# Rendu Final Projet Fairness en IA

Étudiant 01 : MEDJADJ Mohamed Abderraouf <br>
Étudiant 02 : KERMADJ Zineddine <br>
Groupe : 01 <br>
Parcours : LDD3 Magistère d'Informatique

---

## I. Introduction

### a. Objectif du projet :
L’objectif de ce projet est d’**analyser un sous-ensemble de métadonnées et d’images** du NIH Chest X-ray Dataset, comprenant environ 11 000 individus (nos deux dossiers fusionnés), afin d’**identifier d’éventuels biais**. Après avoir appliqué une méthode de prétraitement pour **réduire ces biais et améliorer l’équité des données**, nous entraînerons un **modèle de classification d’images**. Enfin, nous analyserons **l’impact de la pondération sur les performances du modèle** ainsi que **l’effet du post-traitement** sur l’atténuation des biais.
### b. Description du dataset :
*Le NIH Chest X-ray Dataset est un vaste ensemble de données médicales comprenant **112 120 images** de radiographies thoraciques issues de **30 805 patients uniques**, avec des étiquettes de maladies générées par traitement automatique du langage naturel (**NLP**) à partir des rapports radiologiques. Ce jeu de données vise à pallier le **manque d’images médicales annotées**, un obstacle majeur au développement de systèmes de diagnostic assisté par ordinateur (CAD) cliniquement pertinents. Les étiquettes sont estimées à plus de **90 % de précision**, rendant cet ensemble adapté à l’apprentissage faiblement supervisé. Avant sa publication, le plus grand jeu de données disponible comptait seulement 4 143 images. Plus de détails sur l’ensemble de données et le processus d’annotation sont disponibles dans l’article en libre accès : « ChestX-ray8: Hospital-scale Chest X-ray Database and Benchmarks on Weakly-Supervised Classification and Localization of Common Thorax Diseases » (Wang et al.).*
### c. Contenu du dataset :
***Image Index** : Identifiant unique pour chaque image. <br>
**Finding Labels** : Diagnostiques associés à l'image (plusieurs diagnostics peuvent être présents). <br>
**Follow-up** # : Le numéro de suivi, indiquant si l'image appartient à un suivi ou à une première consultation. <br>
**Patient ID** : Identifiant unique pour chaque patient. <br>
**Patient Age** : L'âge du patient. <br>
**Patient Gender** : Le genre du patient. <br>
**View Position** : La position de l'image (par exemple, AP pour antéro-postérieur). <br>
**Dimensions et espacements de l'image** : Ces informations peuvent être utiles pour l'analyse des images, mais elles ne semblent pas directement liées à l'identification des biais.*

### SOMMAIRE:
<pre><b>
I.   Introduction
II.    0. Fonctions Utilitaires
II.    1. Préparation des données
II.    2. Analyse des données
II.    3. Identification des biais
III. Application des méthodes de preprocessing
IV.  Application des méthodes de postprocessing
V.   Analyse et compréhension
VI.  Conclusion
</b></pre>

---
# **REMARQUE IMPORTANTE:**
### *Les explications des sorties, commentaires des graphiques, etc, sont inclus dans des cellules de **code, et pas markdown**, veuillez donc s'il vous plaît ne pas ignorer les lignes commentées.*
---

## II.0. Fonctions utilitaires

### a. Import des librairies nécessaires

In [1]:
from train_classifieur import train_classifier, pred_classifier
import pandas as pd
import numpy as np
import plotly.express as px
import os
from aif360.datasets import BinaryLabelDataset
from aif360.sklearn.metrics import *

2025-04-02 19:56:26.654829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743616586.743088   70007 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743616586.770223   70007 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-02 19:56:26.980588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/raouf/Fairness/myenv/lib/python3.10/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorc

### b. Les fonctions utilitaires

In [2]:
# Fonction pour le calcul des métriques de fairness

def get_group_metrics(
    y_true,
    y_pred=None,
    prot_attr=None,
    priv_group=1,
    pos_label=1,
    sample_weight=None,
):
    group_metrics = {}
    group_metrics["base_rate"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
    return group_metrics

## II.1. Preparation des données

### a. Chargement du dataset

In [3]:
DATA_DIR = "./DATA"
df = pd.read_csv(DATA_DIR+"/metadata.csv")

### b. Exploration préliminaire

In [4]:
print(df.shape)
# Affichons les 5 premiers points de donnée du dataset
df.head()

(10948, 12)


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],WEIGHTS
0,00000028_000.png,Pleural_Thickening,0,28,63,M,PA,2048,2500,0.168000,0.168000,1
1,00000037_000.png,No Finding,0,37,72,M,PA,2708,2638,0.143000,0.143000,1
2,00000044_000.png,Consolidation|Effusion|Infiltration,0,44,79,M,PA,2010,2021,0.194311,0.194311,1
3,00000044_001.png,Infiltration|Pleural_Thickening,1,44,78,M,PA,2544,3056,0.139000,0.139000,1
4,00000044_002.png,Cardiomegaly,2,44,78,M,PA,3056,2544,0.139000,0.139000,1


In [5]:
# Vérifions les types de données et les valeurs manquantes
df.info()

# Explication des sorties:
# Le dataset est composé de 12 colonnes, dont 8 correspondent à des features numériques, et 4 catégorielles.
# Toutes les colonnes ne contiennent pas de valeurs nulles sauf la dernière (Unnamed: 11), qui contient que des valeurs nulles.
# Le dataset contient 54009 points de données.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10948 entries, 0 to 10947
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Image Index                  10948 non-null  object 
 1   Finding Labels               10948 non-null  object 
 2   Follow-up #                  10948 non-null  int64  
 3   Patient ID                   10948 non-null  int64  
 4   Patient Age                  10948 non-null  int64  
 5   Patient Gender               10948 non-null  object 
 6   View Position                10948 non-null  object 
 7   OriginalImage[Width          10948 non-null  int64  
 8   Height]                      10948 non-null  int64  
 9   OriginalImagePixelSpacing[x  10948 non-null  float64
 10  y]                           10948 non-null  float64
 11  WEIGHTS                      10948 non-null  int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 1.0+ MB


In [6]:
# Statistiques descriptives
df.describe()

# Explication des sorties:
# Max: On remarque qu'il y a une valeur max = 412 pour l'age, qui n'est pas normal (outlier), et qui peut être dû à une erreur de frappe.
# Count = nombre de points de données sauf pour 'Unnamed: 11', cette colonne contient que des null.
# Mean: Moyenne des valeurs par colonne, pas de remarque importante.
# Std: Standard deviation des valeurs par colonne, pas de remarque importante.
# Min: Le min des valeurs par colonne, pas de remarque importante.
# Les Quantiles: pas de remarque importante.

,Follow-up #,Patient ID,Patient Age,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],WEIGHTS
count,10948.000000,10948.000000,10948.000000,10948.000000,10948.000000,10948.000000,10948.000000,10948.0
mean,8.047497,14060.749817,45.946840,2655.447662,2493.314487,0.155255,0.155255,1.0
std,12.626621,8588.887031,17.864949,339.587310,401.468117,0.016136,0.016136,0.0
min,0.000000,13.000000,1.000000,1282.000000,1153.000000,0.115000,0.115000,1.0
25%,0.000000,7018.000000,33.000000,2500.000000,2048.000000,0.143000,0.143000,1.0
50%,3.000000,13890.000000,48.000000,2544.000000,2544.000000,0.143000,0.143000,1.0
75%,10.000000,20675.000000,58.000000,2992.000000,2991.000000,0.168000,0.168000,1.0
max,80.000000,30803.000000,412.000000,3451.000000,3056.000000,0.194323,0.194323,1.0


### c. Préparation du dataset

In [7]:
# On sauvegarde le dataframe original avant toute transformation
original_df = df.copy()

In [8]:
# Fonction pour faire des train-test split:

def train_test_split(df):
    train_sain_path = DATA_DIR+"/train/sain"
    train_malade_path = DATA_DIR+"/train/malade"

    train_images = set(os.listdir(train_sain_path) + os.listdir(train_malade_path))

    df["in_train"] = df["Image Index"].apply(lambda x: 1 if x in train_images else 0)

In [9]:
# Élimination des outliers (age > 130 years old)
df = df[df['Patient Age'] <= 130]

# Séparation des données train-test
train_test_split(df)

# Séparation des colonnes liées aux images des autres métadonnées
df_image_related = df[['Image Index', 'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'View Position']]
df_others = df.drop(columns=['Image Index', 
    'OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'View Position'])

# Déplacement du label 'Finding Labels' à la fin du dataframe.
df_others = df_others[[col for col in df_others.columns if col != 'Finding Labels'] + ['Finding Labels']]
df_original_label = df_others[['Finding Labels']]

print(df_others.shape)
# On remarque que le nombre de points de données a diminué de 6 (outliers eliminés).

df_others.head()

(10945, 7)


,Follow-up #,Patient ID,Patient Age,Patient Gender,WEIGHTS,in_train,Finding Labels
0,0,28,63,M,1,0,Pleural_Thickening
1,0,37,72,M,1,1,No Finding
2,0,44,79,M,1,1,Consolidation|Effusion|Infiltration
3,1,44,78,M,1,1,Infiltration|Pleural_Thickening
4,2,44,78,M,1,1,Cardiomegaly


In [10]:
# VERSION 1: encodage one-hot du label

# a. Création d'une colonne 'Finding Labels' contenant une liste des labels
df_encoded_OH = df_others.copy()
df_encoded_OH['Finding Labels'] = df_encoded_OH['Finding Labels'].replace('No Finding', '').str.split('|')
df_encoded_OH['Finding Labels'] = df_encoded_OH['Finding Labels'].apply(lambda x: [] if x == [''] else x)

# b. Encodage des colonnes en One-Hot avec le prefix "Finding_"
all_labels = set([label for sublist in df_encoded_OH['Finding Labels'] for label in sublist])
for label in all_labels:
    column_name = f"Finding_{label.lower().replace(' ', '_')}"
    df_encoded_OH[column_name] = df_encoded_OH['Finding Labels'].apply(lambda x: 1 if label in x else 0)
df_encoded_OH = df_encoded_OH.drop(columns=['Finding Labels'])

df_encoded_OH

,Follow-up #,Patient ID,Patient Age,Patient Gender,WEIGHTS,in_train,Finding_pneumonia,Finding_infiltration,Finding_mass,Finding_nodule,Finding_emphysema,Finding_pneumothorax,Finding_atelectasis,Finding_cardiomegaly,Finding_hernia,Finding_pleural_thickening,Finding_edema,Finding_fibrosis,Finding_consolidation,Finding_effusion
0,0,28,63,M,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,37,72,M,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,44,79,M,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1
3,1,44,78,M,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
4,2,44,78,M,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10943,10,30753,55,F,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1
10944,11,30753,54,F,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
10945,12,30753,55,F,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
10946,0,30782,40,M,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# VERSION 2: encodage binaire (1 ssi patient malade) du label

df_encoded_bool = df_others.copy()
df_encoded_bool['is_ill'] = (df_encoded_bool['Finding Labels'] != 'No Finding').astype(int)
df_encoded_bool = df_encoded_bool.drop(columns=['Finding Labels'])

df_encoded_bool

,Follow-up #,Patient ID,Patient Age,Patient Gender,WEIGHTS,in_train,is_ill
0,0,28,63,M,1,0,1
1,0,37,72,M,1,1,0
2,0,44,79,M,1,1,1
3,1,44,78,M,1,1,1
4,2,44,78,M,1,1,1
...,...,...,...,...,...,...,...
10943,10,30753,55,F,1,1,1
10944,11,30753,54,F,1,1,1
10945,12,30753,55,F,1,1,1
10946,0,30782,40,M,1,1,0


### d. Préparation du dataset AIF360

In [12]:
# VERSION 3: encodage sous format AIF360

protected_attributes = ['Patient Gender']
protected_attribute = protected_attributes[0]

def get_aif360_data(df):
    dff = df.copy()
    dff["Patient Gender"] = dff["Patient Gender"].map({"M": 0, "F": 1})

    ret_df = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff,
        label_names=['is_ill'], 
        protected_attribute_names=protected_attributes
    )

    ret_df_train = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff[dff["in_train"]==1],
        label_names=['is_ill'], 
        protected_attribute_names=protected_attributes
    )

    ret_df_valid = BinaryLabelDataset(
        favorable_label=0,  # "sain" est la classe favorable
        unfavorable_label=1,  # "malade" est la classe défavorable
        df=dff[dff["in_train"]==0],
        label_names=['is_ill'], 
        protected_attribute_names=protected_attributes
    )

    return ret_df, ret_df_train, ret_df_valid

aif_df, aif_df_train, aif_df_valid = get_aif360_data(df_encoded_bool)

aif_df

               instance weights    features                         \
                                                                     
                                Follow-up # Patient ID Patient Age   
instance names                                                       
0                           1.0         0.0       28.0        63.0   
1                           1.0         0.0       37.0        72.0   
2                           1.0         0.0       44.0        79.0   
3                           1.0         1.0       44.0        78.0   
4                           1.0         2.0       44.0        78.0   
...                         ...         ...        ...         ...   
10943                       1.0        10.0    30753.0        55.0   
10944                       1.0        11.0    30753.0        54.0   
10945                       1.0        12.0    30753.0        55.0   
10946                       1.0         0.0    30782.0        40.0   
10947               

## II.2. Analyse des données

## II.3. Identification des biais

---

## III. Application des méthodes de preprocessing

In [17]:
metrics_before_training = get_group_metrics(
    y_true=df_encoded_bool['is_ill'],
    y_pred=None,
    prot_attr=df_encoded_bool[protected_attribute],
    priv_group='M',
    pos_label=0
)

# Affichage des résultats
for metric, value in metrics_before_training.items():
    print(f"{metric}: {value:.4f}")

base_rate: 0.5489
statistical_parity_difference: 0.0429
disparate_impact_ratio: 1.0808


---

## IV. Application des méthodes de postprocessing

### -> Zineddine - insert cells here

---

## V. Analyse et compréhension

### -> Get together and link preproc and postproc results

---

## VI. Conclusion

### *Preferably*, get together and discuss final comments